In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from  sklearn.decomposition import  PCA
from sklearn import preprocessing
from sklearn import tree
from sklearn.manifold import TSNE
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split


pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')
%matplotlib inline 

In [ ]:
# 首次转换为feather格式
data1 = pd.read_csv('../data.csv')
data1.to_feather('data1.feather')

data = pd.read_excel('features1.xlsx')
data.to_feather('features1.feather')

In [ ]:
data_ = pd.read_feather('data1.feather')
class_ = data_['class']
data__ = pd.read_feather('features1.feather')
# 假设你的特征集是 data，目标变量是 class_
features = data__  # 特征集
y_ = class_  # 目标变量

去噪后新文件

In [18]:
demo = pd.read_feather('classes_with_DBSCAN.feather')
class_ = demo['class']
features = demo.drop(columns=['class'])

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(features)
y = pd.Series(class_)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

随机森林

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics  
rf = RandomForestClassifier().fit(x_train,y_train)

# 计算训练和测试集的准确率
train_acc = metrics.accuracy_score(y_train, rf.predict(x_train))
test_acc = metrics.accuracy_score(y_test, rf.predict(x_test))
print(f'Train Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')


随机森林and梯度特征消除

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics  
from sklearn.feature_selection import RFE

rf = RandomForestClassifier()
rfe = RFE(estimator=rf, n_features_to_select=10)
rfe = rfe.fit(x_train, y_train)
# 选择特征
x_train_rfe = rfe.transform(x_train)
x_test_rfe = rfe.transform(x_test)
# 在选择的特征上训练随机森林分类器
rf.fit(x_train_rfe, y_train)

# 计算训练和测试集的准确率
train_acc = metrics.accuracy_score(y_train, rf.predict(x_train_rfe))
test_acc = metrics.accuracy_score(y_test, rf.predict(x_test_rfe))

print(f'Train Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')

# 打印特征排名
print("Feature Ranking:", rfe.ranking_)

Train Accuracy: 1.0
Test Accuracy: 0.9552803577571379
Feature Ranking: [15 70 48 13 85 30  4 46 26 28  6  1  1 53 62 44 42 77 60 73  1  1 16 65
 82 20 83 19 75  3 33 71 76 93  1 67 72 69  1 51 21 36 63  8  9  1 17 78
  1  1 50 81 94 47 11 31 34 68 25 45 88 22 59 14 80 55 29 90 64 74 24 38
 23 56 91 40 87 35 43 49 39 18 58 84 37  2  7 52 89 10 86 61 27 66  5  1
 57 79 12 92 32 41 54]


lda

In [ ]:
lda = LinearDiscriminantAnalysis().fit(x_train, y_train)
train_acc = metrics.accuracy_score(y_train,lda.predict(x_train))
test_acc = metrics.accuracy_score(y_test, lda.predict(x_test))
print(f'Train Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')

k近邻

In [ ]:
knn = KNeighborsClassifier(n_neighbors=30).fit(x_train, y_train)
train_acc = metrics.accuracy_score(y_train,knn.predict(x_train))
test_acc = metrics.accuracy_score(y_test, knn.predict(x_test))
print(f'Train Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')

贝叶斯判别

In [ ]:
gnb = GaussianNB().fit(x_train, y_train)
train_acc_b = metrics.accuracy_score(y_train,gnb.predict(x_train))
test_acc_b = metrics.accuracy_score(y_test, gnb.predict(x_test))
print(f'Train Accuracy: {train_acc_b}')
print(f'Test Accuracy: {test_acc_b}')

决策树

In [11]:
tdt = DecisionTreeClassifier().fit(x_train, y_train)
train_acc1 = metrics.accuracy_score(y_train,tdt.predict(x_train))
test_acc1 = metrics.accuracy_score(y_test, tdt.predict(x_test))
print(f'Train Accuracy: {train_acc1}')
print(f'Test Accuracy: {test_acc1}')

Train Accuracy: 1.0
Test Accuracy: 0.6761477045908184


XGBOOST Classifier

In [ ]:
le = preprocessing.LabelEncoder().fit(y_train)
y_xgb_train = le.transform(y_train)
y_xgb_test = le.transform(y_test)

xgb_params = {'n_estimators': 100,
              'learning_rate': 0.2,
              'subsample': 0.927,
              'colsample_bytree': 0.88,
              'max_depth': 5,
              'booster': 'gbtree', 
              'reg_lambda': 38,
              'reg_alpha': 32,
              'random_state': 12}
xgb = XGBClassifier(**xgb_params).fit(x_train, y_xgb_train)
accuracy1 = metrics.accuracy_score(y_train,xgb.predict(x_train))
accuracy2 = metrics.accuracy_score(y_test, xgb.predict(x_test))
print(f'Train Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')

In [12]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(x_train,y_train)
train_acc_l = metrics.accuracy_score(y_train,logistic.predict(x_train))
test_acc_l = metrics.accuracy_score(y_test, logistic.predict(x_test))
print(f'Train Accuracy: {train_acc_l}')
print(f'Test Accuracy: {test_acc_l}')

Train Accuracy: 0.7448674080410608
Test Accuracy: 0.7360279441117764
